In [1]:
from bokeh.plotting import figure, show
from bokeh.io import output_notebook, export_svg
import pandas as pd
from datetime import datetime
import numpy as np

#Parse str into date (for df['Date])
def toDate(value: str) -> datetime.date:
    return datetime.strptime(value, '%m/%d/%Y')

#Import two separate databases
database_1 = pd.read_csv('/Users/jjmtz/Documents/Programming/Workstation/jupyter-notebook projects/IYW Data Analysis/IYW-database1.csv')
database_2 = pd.read_csv('/Users/jjmtz/Documents/Programming/Workstation/jupyter-notebook projects/IYW Data Analysis/IYW-database2.csv')

#Concatenate the databases and fix indexes
df = pd.concat([database_2, database_1])
df['indx'] = [key for key, _ in enumerate(df['Price'])]
df.index = df['indx']

#Clean temporal values for databases
del database_1, database_2
output_notebook()

#Apply toDate function to the dates, in order to plot the chart in bokeh
df['Date'] = df['Date'].map(toDate)

#Prepare the numeric values from the dates for linear and polynomial regression
dates_for_regression = df['Date'].map(datetime.toordinal)
linear_model = np.poly1d(np.polyfit(dates_for_regression, df['Price'], 1))
polynomical_model = np.poly1d(np.polyfit(dates_for_regression, df['Price'], 2))


#Set the chart info
p = figure(x_axis_type='datetime', title="IYW Index Stock History", width = 1000, height = 600)
p.outline_line_width = 20
p.xaxis.axis_label = 'Date'
p.yaxis.axis_label = 'Value'

#Lines
p.line(x = df['Date'],  y = df['Low'],    color = 'red',    legend_label = 'Low') 
p.line(x = df['Date'],  y = df['High'],   color = 'green',  legend_label = 'High')
p.line(x = df['Date'],  y = df['Price'],  color = 'black',  legend_label = 'Close')

#Linear and Polinomical Regression Fit
p.line(x = df['Date'],  y = linear_model(dates_for_regression), color = 'gray', legend_label = '1° and 2° Fit', dash = 'dashed', alpha = 0.5)
p.line(x = df['Date'],  y = polynomical_model(dates_for_regression), color = 'gray', dash = 'dashed', alpha = 0.5)
p.legend.location = "top_left"

#Show the graph
show(p)

#Prepare for export
p.background_fill_color = None
p.border_fill_color = None
p.output_backend = "svg"
export_svg(p, filename="chart.svg")

Loading BokehJS ...